# 스팀 게임 데이터 분석
- 게임 리뷰 / 뉴스
- 인디게임 개발 트랜드
- 게임 플레이 시간과 리뷰 점수의 상관관계
- 유저 이탈률 예측
- 시간에 따른 게임 개발 트렌드 변화

In [2]:
import os
import sys
import requests
from bs4 import BeautifulSoup
import pprint
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())
from config import keys

api_key = keys.STEAM_API_KEY



In [2]:
# 글로벌 성과 관련 / https://partner.steamgames.com/doc/webapi/ISteamUserStats
url = "http://api.steampowered.com/ISteamUserStats/GetGlobalAchievementPercentagesForApp/v0002/"
params = {
    "key": api_key,
    "gameid": 1778820
}

res = requests.get(url, params=params)
print(res.text)
res_dict = dict(res.json())
print(res_dict)

{"achievementpercentages":{"achievements":[{"name":"HeatBurst","percent":89.5},{"name":"BattleTornado","percent":88.4000015258789063},{"name":"HeatTechnique","percent":87.0999984741210938},{"name":"Practicedamage","percent":85.8000030517578125},{"name":"recoverygauge","percent":84.5},{"name":"BattlePerfect","percent":83.5},{"name":"RageDamage","percent":80},{"name":"BattleGreat","percent":80},{"name":"HeatSmash","percent":75.9000015258789063},{"name":"BattleCombo70","percent":75.8000030517578125},{"name":"Customize","percent":74.5999984741210938},{"name":"OnlineBattle10wins","percent":72.0999984741210938},{"name":"HeatDash","percent":66.5},{"name":"BattleRageArts","percent":66.3000030517578125},{"name":"FightMoney","percent":65.4000015258789063},{"name":"RankMatchWin","percent":64.9000015258789063},{"name":"FloorBlast","percent":64.8000030517578125},{"name":"FloorBreak","percent":64.5999984741210938},{"name":"WallBound","percent":63.9000015258789063},{"name":"Promote1st","percent":61.4

In [4]:
# 유저 리뷰 데이터 / https://partner.steamgames.com/doc/store/getreviews
appid = 620
url = f"https://store.steampowered.com/appreviews/{appid}?json=1"

response = requests.get(url)
response.json()

{'success': 1,
 'query_summary': {'num_reviews': 20,
  'review_score': 9,
  'review_score_desc': 'Overwhelmingly Positive',
  'total_positive': 150881,
  'total_negative': 1755,
  'total_reviews': 152636},
 'reviews': [{'recommendationid': '179221491',
   'author': {'steamid': '76561199697272356',
    'num_games_owned': 0,
    'num_reviews': 1,
    'playtime_forever': 471,
    'playtime_last_two_weeks': 107,
    'playtime_at_review': 364,
    'last_played': 1734252041},
   'language': 'english',
   'review': "Life-changing, first proper story game I've played and a damn good one to start with, couldn't recommend more if i tried.",
   'timestamp_created': 1731838200,
   'timestamp_updated': 1731838200,
   'voted_up': True,
   'votes_up': 53,
   'votes_funny': 1,
   'weighted_vote_score': '0.824981153011322021',
   'comment_count': 0,
   'steam_purchase': True,
   'received_for_free': False,
   'written_during_early_access': False,
   'primarily_steam_deck': False},
  {'recommendationid'

In [ ]:
# 전체 게임 정보 / https://steamspy.com/api.php / https://steamspy.com/about
url = "https://steamspy.com/api.php?request=all&page=1"
response = requests.get(url)
response.json()

{'9450': {'appid': 9450,
  'name': 'Warhammer 40,000: Dawn of War - Soulstorm',
  'developer': 'Relic Entertainment',
  'publisher': 'SEGA',
  'score_rank': '',
  'positive': 16011,
  'negative': 776,
  'userscore': 0,
  'owners': '1,000,000 .. 2,000,000',
  'average_forever': 1890,
  'average_2weeks': 249,
  'median_forever': 598,
  'median_2weeks': 296,
  'price': '0',
  'initialprice': '0',
  'discount': '0',
  'ccu': 991},
 '512900': {'appid': 512900,
  'name': 'Streets of Rogue',
  'developer': 'Matt Dabrowski',
  'publisher': 'tinyBuild',
  'score_rank': '',
  'positive': 20573,
  'negative': 824,
  'userscore': 0,
  'owners': '1,000,000 .. 2,000,000',
  'average_forever': 1212,
  'average_2weeks': 559,
  'median_forever': 227,
  'median_2weeks': 559,
  'price': '399',
  'initialprice': '1999',
  'discount': '80',
  'ccu': 278},
 '313120': {'appid': 313120,
  'name': 'Stranded Deep',
  'developer': 'Beam Team Games',
  'publisher': 'Beam Team Publishing',
  'score_rank': '',
  'p

In [5]:
# 게임 접속자 수 / https://steamcharts.com/
appid = "620"
url = f"https://steamcharts.com/app/{appid}"
response = requests.get(url)
    
if response.status_code != 200:
    print("페이지를 불러올 수 없습니다.")
    
soup = BeautifulSoup(response.text, 'html.parser')
    
# 실시간 동시 접속자 수 가져오기
current_players_section = soup.find("div", {"class": "app-stat"})
current_players = current_players_section.find("span", {"class": "num"}).text.strip()
    
# 월별 사용자 통계 가져오기
table = soup.find("table", {"class": "common-table"})
rows = table.find_all("tr")[1:]  # 헤더 제외
    
data = []
for row in rows:
    cols = row.find_all("td")
    month = cols[0].text.strip()
    avg_players = cols[1].text.strip()
    gain = cols[2].text.strip()
    peak_players = cols[4].text.strip()
    
    data.append({
        "Month": month,
        "Average Players": avg_players,
        "Player Gain": gain,
        "Peak Players": peak_players
    })
data[:5]
# # 데이터프레임 생성
# df = pd.DataFrame(data)

[{'Month': 'Last 30 Days',
  'Average Players': '2066.07',
  'Player Gain': '+273.1',
  'Peak Players': '4831'},
 {'Month': 'November 2024',
  'Average Players': '1792.92',
  'Player Gain': '-707.67',
  'Peak Players': '4831'},
 {'Month': 'October 2024',
  'Average Players': '2500.60',
  'Player Gain': '1057.83',
  'Peak Players': '8711'},
 {'Month': 'September 2024',
  'Average Players': '1442.77',
  'Player Gain': '-366.21',
  'Peak Players': '2655'},
 {'Month': 'August 2024',
  'Average Players': '1808.98',
  'Player Gain': '-1289.75',
  'Peak Players': '3097'}]

In [ ]:
# 게임 가격 정보 / https://steamdb.info/

# # TODO
# # 쿠키 수집
# steamdb_url = f"https://steamdb.info/"

appid = "620"
url = f"https://steamdb.info/api/GetPriceHistory/?appid={appid}&cc=kr"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
    "Accept": "application/json",
    "Referer": f"https://steamdb.info/app/{appid}/",
    "Cookie": "cf_clearance=ZUoklX6fIbVZL79XK4wM_FIJDu6LVk4nt8F3Sf793LM-1734410291-1.2.1.1-_q.7ZNrp1OYP_vJLcW2rBChahG61trZy_HQIebr2U.iTA564uU9fmfqYAhQX093tQln2QJsPfWAVW6jzXKiYFTJeixVUTQUsnYU_BqnkN1J1PFkLOKs5zlkiboVHKUmPQni9DQX4hO3Zd4K_5w4GnSqcpRblaACJz2Hnc8twuVZcIfP82Q6NkKFoHEh5FDgeC99vXge0nI7CLcbnM6G0AfffUBA5SibAFbB_w4m8Ny3VcTwWKWJvO0R.4yqXk6KifRi90MZXch3RnTyQdmDZy7d3feTfJNWq3.WiMUZGlDjwNcfLidp4Ryd5QbUJBds7AkN8Ugx93ux_M55A3e49MuYrkxI5S6J5p2CkBMHdIndVBqj_oaYp7luEDJpdjbtGySfvTChBlF5L6l28IKGzjShsH2M.CXwhLd0cAIM_HqtOHU3b78LP_t6SHYg7yKS_; __cf_bm=tRIj6dWW2g9XMLdQorWHANDL49tJcG9MUQtSxPFjxi4-1734424200-1.0.1.1-V6AQQjkPoB9ACjfPDONQa_qV7SRbBM1_QaduU5uD_ltgsLVms_j8pTladIZgRFBlXSMYE4aBc0paQabAX8pI.w",
    "X-Requested-With": "XMLHttpRequest",
}

# 페이지 요청
response = requests.get(url, headers=headers)
if response.status_code != 200:
    print(f"페이지 요청 실패: 상태 코드 {response.status_code}")

response.json()

{'success': True,
 'data': {'history': [{'x': 1671732494000, 'y': 1100, 'f': '₩ 1100', 'd': 90},
   {'x': 1672942886000, 'y': 11000, 'f': '₩ 11000', 'd': 0},
   {'x': 1677522292000, 'y': 1100, 'f': '₩ 1100', 'd': 90},
   {'x': 1678126880000, 'y': 11000, 'f': '₩ 11000', 'd': 0},
   {'x': 1678988050000, 'y': 1100, 'f': '₩ 1100', 'd': 90},
   {'x': 1679592077000, 'y': 11000, 'f': '₩ 11000', 'd': 0},
   {'x': 1682356878000, 'y': 1100, 'f': '₩ 1100', 'd': 90},
   {'x': 1682961679000, 'y': 11000, 'f': '₩ 11000', 'd': 0},
   {'x': 1688059498000, 'y': 1100, 'f': '₩ 1100', 'd': 90},
   {'x': 1689268877000, 'y': 11000, 'f': '₩ 11000', 'd': 0},
   {'x': 1691082077000, 'y': 1100, 'f': '₩ 1100', 'd': 90},
   {'x': 1691687104000, 'y': 11000, 'f': '₩ 11000', 'd': 0},
   {'x': 1694538077000, 'y': 1100, 'f': '₩ 1100', 'd': 90},
   {'x': 1695142879000, 'y': 11000, 'f': '₩ 11000', 'd': 0},
   {'x': 1700590100000, 'y': 1100, 'f': '₩ 1100', 'd': 90},
   {'x': 1701195982000, 'y': 11000, 'f': '₩ 11000', 'd':

In [81]:
print(table)
print(len(rows))
data[:5]

<table class="table table-hover" hidden="" id="chart-month-table">
<thead>
<tr>
<th class="sort-default sort-down" style="width:200px">Month</th>
<th>Peak</th>
<th>Gain</th>
<th>% Gain</th>
<th>Average</th>
<th>Avg % Gain</th>
</tr>
</thead>
<tbody class="tabular-nums"></tbody>
</table>
1


[]